
## DataFrame進階用法

在第一部分時我們介紹了如何在dataframe中挑選、排序、增加/刪除欄位、以及合併兩個或多個資料表，然而Pandas的功能還萬萬不只這樣。這個部份我們將會額外介紹三個好用的方法，分別為apply, groupby, 以及pivot_table。

- ### apply功能的使用

相對而言apply的使用方式並沒有那麼的直觀，但你可以想像它是另一種形式的迴圈，它可以依照每個小分組(通常是一欄或一列，但我們也可以搭配著groupby使用)重複地執行同一個函數，例如在以下的例子當中，我們使用apply去計算Age與Fare的平均數是多少，然而apply更強大的功能是可以放入自己定義的函數做運算。

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
# 此步驟需要 google 權限認證
%matplotlib inline

# 載入範例的資料集，這次我們使用的是鐵達尼號的資料，後續在機器學習領域上會很常見到這份資料集, 將資料集指到 google drive 底下 My Drive/Python-Data-Analysis-master/dataset/titanic/train.csv, 如有更動此預設位子請在自行調整
df = pd.read_csv('/content/drive/My Drive/Python-Data-Analysis-master/dataset/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 挑選出Age與Fare欄位並且用apply計算每個欄位的平均
# 還記得axis的用法嗎? axis=0代表我們將第一個軸(row)的數值視為同一群，第二個軸(column)的數值則分開計算
df[['Age', 'Fare']].apply(np.mean, axis=0)

Age     29.699118
Fare    32.204208
dtype: float64

In [5]:
# 挑選出Age與Fare欄位並且以apply計算每筆資料的平均
# 在此我們設定axis=1去計算每筆資料在這兩個欄位的平均，雖然這個數值在解釋上並無任何意義
df[['Age', 'Fare']].apply(np.mean, axis=1).head()

0    14.62500
1    54.64165
2    16.96250
3    44.05000
4    21.52500
dtype: float64

In [6]:
# 我們也可以自行定義一個函數並且使用apply去做計算
# 在此我們使用年齡與性別去決定每個乘客的身分


def get_identity(df):
    if df['Sex'] == 'male':
        if df['Age'] < 18:
            return 'Child'
        else:
            return 'Man'
    elif df['Sex'] == 'female':
        if df['Age'] < 18:
            return 'Child'
        else:
            return 'Woman'


# 依據每個row執行get_identity這個函數，並將結果儲存至新的identity欄位
df['identity'] = df.apply(get_identity, axis=1)

# 觀察前20筆資料的結果
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,identity
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Man
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Woman
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Woman
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Woman
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Man
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Man
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Man
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Child
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Woman
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Child


- ### 群組的使用

在比較進階的資料處理上，我們會希望針對每個不同的類別分別了解他們的特性，例如希望依據縣市計算出各區域的人民平均月薪，這個時候groupby就可以派上用場。在此我們同樣以鐵達尼號的乘客資料進行示範。

In [7]:
# 依照上船港口(Embarked)欄位做群組，挑選出票價並且計算平均
df.groupby(by='Embarked')['Fare'].mean()

Embarked
C    59.954144
Q    13.276030
S    27.079812
Name: Fare, dtype: float64

In [8]:
# 依照上船港口(Embarked)與性別(Sex)計算各欄位的資料數量(如果欄位內沒有遺漏值的話可以將這個數量作為人次來看)
df.groupby(by=['Embarked', 'Sex']).count()

PassengerId  Survived  Pclass  Name  Age  SibSp  Parch  \
Embarked Sex                                                              
C        female           73        73      73    73   61     73     73   
         male             95        95      95    95   69     95     95   
Q        female           36        36      36    36   12     36     36   
         male             41        41      41    41   16     41     41   
S        female          203       203     203   203  186    203    203   
         male            441       441     441   441  368    441    441   

                 Ticket  Fare  Cabin  identity  
Embarked Sex                                    
C        female      73    73     37        73  
         male        95    95     32        95  
Q        female      36    36      2        36  
         male        41    41      2        41  
S        female     203   203     56       203  
         male       441   441     73       441

除了直接使用groupby搭配常見的統計值之外，我們同樣也可以在groupby後加上apply做運算，此時apply會將指定欄位中類別相同的資料視為同一組做一次運算。

In [9]:
# 自定義計算資料中遺漏值總數的函數

def total_na_number(df):
    return df.isna().sum()

# 依據艙等(Pclass)欄位將資料分組，並分別計算各欄位的遺漏數總數
df.groupby(by='Pclass').apply(total_na_number)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,identity
Pclass,,,,,,,,,,,,,
1,0,0,0,0,0,30,0,0,0,0,40,2,0
2,0,0,0,0,0,11,0,0,0,0,168,0,0
3,0,0,0,0,0,136,0,0,0,0,479,0,0


- ### pivot_table的使用

在這邊要講的最後一個函數叫做pivot_table，在功能上對應的是excel中的樞紐分析表，因此它可以彈性地以你想要的方式合併不同組別的資料並且整理成新的表格樣式。

In [10]:
# 以pivot_table方法將資料做總別，設定性別(Sex)的各個類別為列、艙等(Pclass)的各個類別為欄、並且計算每個組別下票價(Fare)的平均。
df.pivot_table(values='Fare', index='Sex', columns='Pclass', aggfunc='mean')

Pclass,1,2,3
Sex,,,
female,106.125798,21.970121,16.118810
male,67.226127,19.741782,12.661633
